In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import glob 
import json 
import shutil
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
splits=['train','test','valid']

path='/Users/chrissu/Desktop/research_data/SARscope'
os.mkdir(path)
for spl in splits:
    try: 
        os.mkdir(path+spl) 
    except OSError as error: 
        print(error)

FileExistsError: [Errno 17] File exists: '/Users/chrissu/Desktop/research_data/SARscope'

In [10]:
def extract_images(source_path,dst_path):
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)
    image_path=glob.glob(source_path+"/*.jpg")
    for path in image_path:
        shutil.copy(path,dst_path)

In [11]:
source_path="/Users/chrissu/Desktop/research_data/SARscope/train"
dst_path="/Users/chrissu/Desktop/research_data/SARscope/train/images/"
extract_images(source_path,dst_path)

In [12]:
source_path="/Users/chrissu/Desktop/research_data/SARscope/test"
dst_path="/Users/chrissu/Desktop/research_data/SARscope/test/images/"
extract_images(source_path,dst_path)

In [13]:
source_path="/Users/chrissu/Desktop/research_data/SARscope/valid"
dst_path="/Users/chrissu/Desktop/research_data/SARscope/valid/images/"
extract_images(source_path,dst_path)

In [14]:
from pylabel import importer
def convert_coco2Yolo(source_path,dst_path):
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)
    json_file=glob.glob(source_path+"/*.json")
    dataset=importer.ImportCoco(json_file[0], path_to_images=source_path, name="SARscope")
    dataset.export.ExportToYoloV5(dst_path)

In [15]:
source_path="/Users/chrissu/Desktop/research_data/SARscope/train"
dst_path="/Users/chrissu/Desktop/research_data/SARscope/train/labels/"
extract_images(source_path,dst_path)

In [16]:
source_path="/Users/chrissu/Desktop/research_data/SARscope/test"
dst_path="/Users/chrissu/Desktop/research_data/SARscope/test/labels/"
extract_images(source_path,dst_path)

In [17]:
source_path="/Users/chrissu/Desktop/research_data/SARscope/valid"
dst_path="/Users/chrissu/Desktop/research_data/SARscope/valid/labels/"
extract_images(source_path,dst_path)

In [25]:
config_file_template = '''
train: /Users/chrissu/Desktop/research_data/SARscope/train/images
val: /Users/chrissu/Desktop/research_data/SARscope/valid/images
test: /Users/chrissu/Desktop/research_data/SARscope/test/images

nc: 2
names: ['background','ship']
'''

with open('/Users/chrissu/Desktop/research_data/SARscope/dataset.yaml', 'w') as f:
    f.write(config_file_template)

In [31]:
import yaml 
with open('/Users/chrissu/Desktop/research_data/SARscope/dataset.yaml', 'r') as f:
    file_path=yaml.safe_load(f)
print(file_path)

{'train': '/Users/chrissu/Desktop/research_data/SARscope/train/images', 'val': '/Users/chrissu/Desktop/research_data/SARscope/valid/images', 'test': '/Users/chrissu/Desktop/research_data/SARscope/test/images', 'nc': 2, 'names': ['background', 'ship']}


In [32]:
from ultralytics import YOLO
model = YOLO('yolov8n.yaml').load('yolov8n.pt')

Transferred 355/355 items from pretrained weights


In [33]:
model.train(data="/Users/chrissu/Desktop/research_data/SARscope/dataset.yaml",epochs=15,imgsz=640,device='cpu')

Ultralytics YOLOv8.2.31 🚀 Python-3.10.14 torch-2.3.1 CPU (Apple M2)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/Users/chrissu/Desktop/research_data/SARscope/dataset.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_c

train: Scanning /Users/chrissu/Desktop/research_data/SARscope/train/labels... 0 images, 4717 backgrounds, 0 corr

train: WARNING ⚠️ No labels found in /Users/chrissu/Desktop/research_data/SARscope/train/labels.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
train: New cache created: /Users/chrissu/Desktop/research_data/SARscope/train/labels.cache
WARNING ⚠️ No labels found in /Users/chrissu/Desktop/research_data/SARscope/train/labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.



val: Scanning /Users/chrissu/Desktop/research_data/SARscope/valid/labels... 0 images, 1346 backgrounds, 0 corrup

val: WARNING ⚠️ No labels found in /Users/chrissu/Desktop/research_data/SARscope/valid/labels.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
val: New cache created: /Users/chrissu/Desktop/research_data/SARscope/valid/labels.cache
WARNING ⚠️ No labels found in /Users/chrissu/Desktop/research_data/SARscope/valid/labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


Plotting labels to /Users/chrissu/.pyenv/runs/detect/train2/labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /Users/chrissu/.pyenv/runs/detect/train2
Starting training for 15 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/15         0G          0      116.6          0          0        640:  19%|█▊        | 55/295 [09:02<39


KeyboardInterrupt: 

In [ ]:
model.export()